In [7]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.neighbors import KNeighborsClassifier

# تعريف پارامتر هاي LBP
radius = 1
n_points = 8
method = 'uniform'

# خواندن عكس ها و تبديل آن ها به خروجي گري اسكيل
img1 = cv2.imread('normal.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('asthma.jpg', cv2.IMREAD_GRAYSCALE)
img3 = cv2.imread('n1.jpg', cv2.IMREAD_GRAYSCALE)

# محاسبه LBP براي هر عكس
lbp1 = local_binary_pattern(img1, n_points, radius, method)
lbp2 = local_binary_pattern(img2, n_points, radius, method)
lbp3 = local_binary_pattern(img3, n_points, radius, method)

# محاسبه بردار ويژگي براي هر عكس با استفاده از توزيع تاريخچه
hist1, _ = np.histogram(lbp1.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
hist1 = hist1.astype('float')
hist1 /= (hist1.sum() + 1e-7)
hist2, _ = np.histogram(lbp2.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
hist2 = hist2.astype('float')
hist2 /= (hist2.sum() + 1e-7)
hist3, _ = np.histogram(lbp3.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
hist3 = hist3.astype('float')
hist3 /= (hist3.sum() + 1e-7)

# تعريف داده هاي آموزش و برچسب هاي آن ها
X_train = np.array([hist1, hist2])
y_train = np.array([0, 1])

# تعريف داده هاي آزمون
X_test = np.array([hist3])

# ايجاد و آموزش يك دسته بندي kNN با k=1
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

# پيش بيني كلاس عكس جديد با استفاده از دسته بندي kNN
y_pred = knn.predict(X_test)

# چاپ نتيجه پيش بيني
if y_pred == 0:
    print('The new image is normal.')
else:
    print('The new image has asthma.')


The new image has asthma.


In [8]:
import cv2
import numpy as np

# تعریف تابع LBP برای محاسبه الگوهای LBP
def LBP(image):
    # تعریف جدول افزونگی باینری LBP
    LBP_table = np.array([[1, 2, 4], [8, 0, 16], [32, 64, 128]], dtype=np.uint8)

    # تبدیل تصویر ورودی به تصویر خاکستری
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # ابعاد تصویر
    height, width = gray_image.shape

    # تصویر LBP خروجی
    LBP_image = np.zeros((height, width), dtype=np.uint8)

    # محاسبه LBP برای هر پیکسل
    for i in range(1, height - 1):
        for j in range(1, width - 1):
            # مقدار مرکزی پیکسل
            center = gray_image[i, j]
            code = 0
            # محاسبه مقدار LBP با استفاده از جدول افزونگی باینری
            for a in range(-1, 2):
                for b in range(-1, 2):
                    if gray_image[i + a, j + b] >= center:
                        code += LBP_table[a + 1, b + 1]
            LBP_image[i, j] = code

    return LBP_image

# خواندن تصویر پزشکی
medical_image = cv2.imread('pa.jpg')

# محاسبه الگوهای LBP
lbp_result = LBP(medical_image)

# نمایش تصویر الگوهای LBP
cv2.imshow('LBP Image', lbp_result)
cv2.waitKey(0)
cv2.destroyAllWindows()


#تشخیص استخوان